# LR-Zerlegung

In [1]:
import numpy as np
import numpy.linalg as la
import numpy.random as rd

## Testdaten

In [2]:
rd.seed (1)

In [3]:
def RandMat (m, n):
    max = 10
    min = - max
    return np.round (rd.rand (m, n) * (max - min) + min, 0)

In [4]:
def pr (mat):
    print (np.round (mat, 3))

In [5]:
class Test:
    n = 3
    A = RandMat (n, n)
    x = RandMat (n, 1)
    b = A @ x

pr (Test.A)
pr (Test.b)
pr (Test.x)
pr (la.solve (Test.A, Test.b))

[[ -2.   4. -10.]
 [ -4.  -7.  -8.]
 [ -6.  -3.  -2.]]
[[-50.]
 [-22.]
 [ -8.]]
[[ 1.]
 [-2.]
 [ 4.]]
[[ 1.]
 [-2.]
 [ 4.]]


## LR-Zerlegung mit Eliminationsmatrizen

In [6]:
def LRmat (A):
    
    n = A.shape [0]
    L = np.eye (n)
    R = np.copy (Test.A)

    pr (L)
    pr (R)

    for i in range (n - 1):
        for k in range (i + 1, n):
            E = np.eye (n); E [k, i] = - R [k, i] / R [i, i]
            R = E @ R
            L = L @ la.inv (E)
            print ('------------------------------', i, k, '------------------------------')
            pr (E)
            pr (L)
            pr (R)

    print ('------------------------------------------------------------')
    pr (A)
    pr (L)
    pr (R)
    pr (L @ R)

LRmat (Test.A)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[ -2.   4. -10.]
 [ -4.  -7.  -8.]
 [ -6.  -3.  -2.]]
------------------------------ 0 1 ------------------------------
[[ 1.  0.  0.]
 [-2.  1.  0.]
 [ 0.  0.  1.]]
[[1. 0. 0.]
 [2. 1. 0.]
 [0. 0. 1.]]
[[ -2.   4. -10.]
 [  0. -15.  12.]
 [ -6.  -3.  -2.]]
------------------------------ 0 2 ------------------------------
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-3.  0.  1.]]
[[1. 0. 0.]
 [2. 1. 0.]
 [3. 0. 1.]]
[[ -2.   4. -10.]
 [  0. -15.  12.]
 [  0. -15.  28.]]
------------------------------ 1 2 ------------------------------
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]]
[[1. 0. 0.]
 [2. 1. 0.]
 [3. 1. 1.]]
[[ -2.   4. -10.]
 [  0. -15.  12.]
 [  0.   0.  16.]]
------------------------------------------------------------
[[ -2.   4. -10.]
 [ -4.  -7.  -8.]
 [ -6.  -3.  -2.]]
[[1. 0. 0.]
 [2. 1. 0.]
 [3. 1. 1.]]
[[ -2.   4. -10.]
 [  0. -15.  12.]
 [  0.   0.  16.]]
[[ -2.   4. -10.]
 [ -4.  -7.  -8.]
 [ -6.  -3.  -2.]]


## LR-Zerlegung ausprogrammiert

### Zerlegung

In [7]:
def LRdecomp1 (A):

    n = A.shape [0]
    L = np.zeros ((n, n))
    R = np.copy (A)

    for i in range (n):
        L [i, i] = 1
        for k in range (i + 1, n):
            t = R [k, i] / R [i, i]
            L [k, i] = t
            for j in range (i, n):
                R [k, j] -= t * R [i, j]

    return L, R

In [8]:
def TestLRdecomp1 (A):
    L, R = LRdecomp1 (A)
    pr (A)
    pr (L)
    pr (R)
    pr (L @ R)

TestLRdecomp1 (Test.A)

[[ -2.   4. -10.]
 [ -4.  -7.  -8.]
 [ -6.  -3.  -2.]]
[[1. 0. 0.]
 [2. 1. 0.]
 [3. 1. 1.]]
[[ -2.   4. -10.]
 [  0. -15.  12.]
 [  0.   0.  16.]]
[[ -2.   4. -10.]
 [ -4.  -7.  -8.]
 [ -6.  -3.  -2.]]


### Lösung des Gleichungssystems

$\begin{align*}
A \, x &\;=\; b \\
L \, R \, x &\;=\; b \\[6pt]
R \, x &\;=\; y \\
L \, y &\;=\; b
\end{align*}$

In [9]:
def LRsolve1 (L, R, b):
    
    n = L.shape [0]

    y = np.empty ((n, 1))
    for i in range (n):
        t = b [i, 0]
        for k in range (0, i):
            t -= L [i, k] * y [k, 0]
        y [i, 0] = t

    x = np.empty ((n, 1))
    for i in range (n - 1, -1, -1):
        t = y [i, 0]
        for k in range (i + 1, n):
            t -= R [i, k] * x [k, 0]
        x [i, 0] = t / R [i, i]

    return x

In [10]:
def TestLRsolve1 (A, b):
    L, R = LRdecomp1 (A)
    x = LRsolve1 (L, R, b)
    pr (x)
    pr (A @ x)
    pr (b)

TestLRsolve1 (Test.A, Test.b)

[[ 1.]
 [-2.]
 [ 4.]]
[[-50.]
 [-22.]
 [ -8.]]
[[-50.]
 [-22.]
 [ -8.]]


## LR-Zerlegung speichereffizient

### Zerlegung

In [11]:
def LRdecomp2 (A):

    n = A.shape [0]
    LR = np.copy (A)

    for i in range (n):
        for k in range (i + 1, n):
            t = LR [k, i] / LR [i, i]
            LR [k, i] = t
            for j in range (i + 1, n):
                LR [k, j] -= t * LR [i, j]

    return LR

In [12]:
def TestLRdecomp2 (A):
    L, R = LRdecomp1 (A)
    pr (L)
    pr (R)
    LR = LRdecomp2 (A)
    pr (LR)

TestLRdecomp2 (Test.A)

[[1. 0. 0.]
 [2. 1. 0.]
 [3. 1. 1.]]
[[ -2.   4. -10.]
 [  0. -15.  12.]
 [  0.   0.  16.]]
[[ -2.   4. -10.]
 [  2. -15.  12.]
 [  3.   1.  16.]]


### Lösung

In [13]:
def LRsolve2 (LR, b):
    
    n = LR.shape [0]

    y = np.empty ((n, 1))
    for i in range (n):
        t = b [i, 0]
        for k in range (0, i):
            t -= LR [i, k] * y [k, 0]
        y [i, 0] = t

    x = np.empty ((n, 1))
    for i in range (n - 1, -1, -1):
        t = y [i, 0]
        for k in range (i + 1, n):
            t -= LR [i, k] * x [k, 0]
        x [i, 0] = t / LR [i, i]

    return x

In [14]:
def TestLRsolve2 (A, b):
    LR = LRdecomp2 (A)
    x = LRsolve2 (LR, b)
    pr (x)
    pr (A @ x)
    pr (b)

TestLRsolve2 (Test.A, Test.b)

[[ 1.]
 [-2.]
 [ 4.]]
[[-50.]
 [-22.]
 [ -8.]]
[[-50.]
 [-22.]
 [ -8.]]
